In [1]:
%%capture
!pip install datasets --no--build--isolation
!pip install seqeval
!pip install transformers[torch]

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForTokenClassification,TrainingArguments,Trainer,DataCollatorForTokenClassification
import torch
import numpy as np
from datasets import load_metric

In [4]:
# loading the dataset
dataset = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [7]:
dataset["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [8]:
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [9]:
# model checkpoint
model_checkpoint = 'bert-base-cased'
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples["tokens"],truncation = True,is_split_into_words = True)
  labels = []
  for i,label in enumerate(examples["ner_tags"]):
    # getting word ids
    word_ids = tokenized_inputs.word_ids(batch_index = i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [11]:
tokenized_dataset = dataset.map(tokenize_and_align_labels,batched = True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [12]:
# loading the pretrained model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,num_labels = 9)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def compute_metric(p):
  predictions,labels = p
  predictions = np.argmax(predictions,axis = 1) # predictions will be a tensor of two dimensional so axis = 1
  # now both predictions and labels have same dimension
  # now i will be creating the true labels by skipping special tokens and subword tokens ##

  true_labels = [
      [label_list[l] for l in label if l!= -100]
      for label in labels
  ]

  true_predictions = [
      [label_list[p] for (p,l) in zip(prediction,label) if l!= -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = metric.compute(predictions = true_predictions, references = true_labels)
  return {
      "precision": results["overall_precision"],
      "recall":results["overall_recall"],
      "f1":results["overall_f1"],
      "accuracy":results["overall_accuracy"]
  }

In [ ]:
!pip install transformers datasets
import numpy as np
from transformers import EvalPrediction

def compute_metric(p: EvalPrediction):
    print(f"Shape of predictions: {p.predictions.shape}")  # Print the shape of predictions

    # Check if predictions has more than 2 dimensions
    if len(p.predictions.shape) > 2:
      predictions = np.argmax(p.predictions, axis=2)
    else:
      # Handle the case where predictions has only 2 dimensions
      predictions = np.argmax(p.predictions, axis=1)

    # now both predictions and labels have same dimension
    # now i will be creating the true labels by skipping special tokens and subword tokens ##

    return {"accuracy": 0.5} # This is a temporary value. Remove this line when you add your metrics calculation.

In [21]:
!pip install transformers datasets seqeval
import numpy as np
from transformers import EvalPrediction
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metric(p: EvalPrediction):
    print(f"Shape of predictions: {p.predictions.shape}")

    if len(p.predictions.shape) > 2:
        predictions = np.argmax(p.predictions, axis=2)
    else:
        predictions = np.argmax(p.predictions, axis=1)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, p.label_ids)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, p.label_ids)
    ]

    results = {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

    return results

In [34]:
label_list = dataset["train"].features["ner_tags"].feature.names

In [15]:
# setup datacollator for dynamic padding
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

In [16]:
# setup the training arguments
training_args = TrainingArguments(
    output_dir = './results',
    eval_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
)

In [22]:
# initializing the trainer with compute matrix and datacollator
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metric,
    )

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.014900,0.038141,0.991180,0.945796,0.948502,0.947147
2,0.009800,0.039813,0.991375,0.947950,0.953214,0.950575


Shape of predictions: (3250, 151, 9)
Shape of predictions: (3250, 151, 9)


TrainOutput(global_step=1756, training_loss=0.010910771566534368, metrics={'train_runtime': 342.8609, 'train_samples_per_second': 81.905, 'train_steps_per_second': 5.122, 'total_flos': 701093358405576.0, 'train_loss': 0.010910771566534368, 'epoch': 2.0})

In [25]:
results = trainer.evaluate()
print(results)

Shape of predictions: (3250, 151, 9)
{'eval_loss': 0.039812635630369186, 'eval_accuracy': 0.9913749464584712, 'eval_precision': 0.9479497907949791, 'eval_recall': 0.9532144059239314, 'eval_f1': 0.950574809096249, 'eval_runtime': 11.4342, 'eval_samples_per_second': 284.236, 'eval_steps_per_second': 17.841, 'epoch': 2.0}


In [26]:
results

{'eval_loss': 0.039812635630369186,
 'eval_accuracy': 0.9913749464584712,
 'eval_precision': 0.9479497907949791,
 'eval_recall': 0.9532144059239314,
 'eval_f1': 0.950574809096249,
 'eval_runtime': 11.4342,
 'eval_samples_per_second': 284.236,
 'eval_steps_per_second': 17.841,
 'epoch': 2.0}

In [27]:
from transformers import AutoTokenizer
import torch

# Tokenize the input sentence
sentence = "John lives in New York and works for Google."

# Assuming you have already loaded your model and tokenizer
# Example: model = YourTrainedModel(), tokenizer = AutoTokenizer.from_pretrained("your-model-name")

def ner_on_sentence(sentence, model, tokenizer):
    # Tokenize the sentence and prepare it for the model
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

    # Get the model's predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted token ids (argmax over the last dimension)
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predictions to label names (assuming label_list is the list of labels from training)
    label_list = model.config.id2label  # Assuming you have this mapping from ids to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [label_list[p.item()] for p in predictions[0]]

    # Print the tokens and their corresponding predicted labels
    for token, label in zip(tokens, predicted_labels):
        print(f"{token}: {label}")

# Call the function with your sentence
ner_on_sentence(sentence, model, tokenizer)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [39]:
from transformers import AutoTokenizer
import torch

# Example sentence
sentence = "SRK lives in Bombay."

# Load your tokenizer and model
# Example: tokenizer = AutoTokenizer.from_pretrained("your-model-name")
# model = YourTrainedModel()

def ner_on_sentence(sentence, model, tokenizer):
    # Tokenize the sentence and prepare it for the model
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

    # Move inputs to the same device as the model
    device = next(model.parameters()).device  # Get the model's device
    inputs = inputs.to(device) # Move the input tensors to the GPU

    # Get the model's predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted token ids (argmax over the last dimension)
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predictions to label names
    label_list = dataset["train"].features["ner_tags"].feature.names  # This maps ids to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [label_list[p.item()] for p in predictions[0]]

    # Remove tokens that are just padding or special tokens
    cleaned_tokens = []
    cleaned_labels = []
    for token, label in zip(tokens, predicted_labels):
        if token not in tokenizer.all_special_tokens:  # Ignore special tokens like [CLS], [SEP], etc.
            cleaned_tokens.append(token)
            cleaned_labels.append(label)

    # Print the tokens and their corresponding predicted labels
    for token, label in zip(cleaned_tokens, cleaned_labels):
        print(f"{token}: {label}")

# Call the function with your sentence
ner_on_sentence(sentence, model, tokenizer)

SR: B-PER
##K: O
lives: O
in: O
Bombay: B-LOC
.: O
